# Titanic - Treinamento

Criar um algorítimo de Machine Learning que possa prever qual passageiro irá sobreviver ou não no desastre.

In [1]:
import pandas as pd
import numpy as np
import time

## Preparação dos dados para análise

In [2]:
train = pd.read_csv('train.csv', index_col=0)
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Boa parte do procedimento de limpeza e tratamento dos dados foram retirados de https://triangleinequality.wordpress.com/2013/09/08/basic-feature-engineering-with-the-titanic-data/. Deste modo, durante o procedimento abaixo, quando pertencer ao site, estes serão identifcados como <b>site</b> no texto ou como comentário na célula.

<b>Site -</b> Reduzindo o nome dos passageiros para os títulos: Mrs, Miss, Mr and Master.

In [3]:
#import string

Não foi utilizado a função string, como no site, pois o módulo find() está depreciado. Deste modo foi substituído pelo módulo _\_contains__.

In [4]:
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if big_string == np.nan:
            return np.nan
        elif big_string.__contains__(substring):
            return substring
    print(big_string)
    return np.nan

In [5]:
# lista de todos os títulos encontrados
title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev', 'Dr', 'Ms', 'Mlle','Col', 'Capt', 'Mme', 'Countess',
                    'Don', 'Jonkheer']

In [6]:
# criando uma nova coluna apenas com os títulos
train['Title'] = train['Name'].map(lambda x: substrings_in_string(x, title_list))

In [7]:
# conferindo a criação da coluna
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [8]:
# conferindo quais são os valores da coluna criada
train.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Mlle', 'Col', 'Capt', 'Countess', 'Jonkheer'],
      dtype=object)

Para finalizar serão reduzidos os nomes apenas para os desejados no início: Mrs, Miss, Mr e Master.

In [9]:
 def replace_titles(x):
        title = x['Title']
        if title in ['Don', 'Major', 'Capt', 'Jonkheer', 'Rev', 'Col']:
            return 'Mr'
        elif title in ['Countess', 'Mme']:
            return 'Mrs'
        elif title in ['Mlle', 'Ms']:
            return 'Miss'
        elif title =='Dr':
            if x['Sex']=='Male':
                return 'Mr'
            else:
                return 'Mrs'
        else:
            return title

In [10]:
train['Title'] = train.apply(replace_titles, axis=1)
train.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master'], dtype=object)

<b>Site -</b> Elimiando os números das cabines.

In [11]:
# lista dos decks
cabin_list = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']

Unknown está definido como NaN. Primeiramente deve-se substituir estes valores.

In [12]:
# preenchendo NaN com Unknown
train.Cabin.fillna('Unknown', inplace=True)

In [13]:
# elimiando os números das cabines
train['Deck'] = train['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))

In [14]:
# confirmando substituição
train.Deck.unique()

array(['Unknown', 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

<b>Site -</b> Criando um tamanho para a família.

In [15]:
# irá se definir como o tamanho a soma das colunas SibSp e Parch
train['FamilySize'] = train.SibSp + train.Parch

<b>Site -</b> Criando um termo de interação entre as variáveis Age e Class como um produto entre elas.

In [16]:
train['AgeClass'] = train.Age*train.Pclass

<b>Site -</b> Criando uma variável de tarifas por passageiro.

In [17]:
train['FarePerPerson'] = train.Fare/(train.FamilySize + 1)

<b>Deste modo encerra-se o tratamento dos dados sugerido pelo site.</b>

Vamos dar uma olhada como finalizou a nossa tabela.

In [18]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,Deck,FamilySize,AgeClass,FarePerPerson
PassengerId,,,,,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Unknown,S,Mr,Unknown,1,66.0,3.62500
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,C,1,38.0,35.64165
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Unknown,S,Miss,Unknown,0,78.0,7.92500
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,C,1,35.0,26.55000
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Unknown,S,Mr,Unknown,0,105.0,8.05000


<ul>
    <li>As colunas Name e Cabin foram substituidas por Title e Deck, podendo ser retiradas da tabela.</li>
    <li>A coluna Ticket não traz nenhum informação.</li>
</ul>

In [19]:
train.drop(labels=['Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

In [20]:
train.dtypes

Survived           int64
Pclass             int64
Sex               object
Age              float64
SibSp              int64
Parch              int64
Fare             float64
Embarked          object
Title             object
Deck              object
FamilySize         int64
AgeClass         float64
FarePerPerson    float64
dtype: object

In [21]:
train.Sex.unique()

array(['male', 'female'], dtype=object)

In [22]:
train.Sex = train.Sex.map({'male': 0, 'female': 1}).astype(int)

In [23]:
train.Embarked.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [24]:
train.Embarked = train.Embarked.map({np.nan: 0, 'S': 1, 'C': 2, 'Q': 3}).astype(int)

In [25]:
train.Title.unique()

array(['Mr', 'Mrs', 'Miss', 'Master'], dtype=object)

In [26]:
train.Title = train.Title.map({'Mr': 0, 'Mrs': 1, 'Miss': 2, 'Master': 3}).astype(int)

In [27]:
train.Deck.unique()

array(['Unknown', 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [28]:
train.Deck = train.Deck.map({'Unknown': 0, 'C': 1, 'E': 2, 'G': 3, 'D': 4, 'A': 5, 'B': 6, 'F': 7, 'T': 8}).astype(int)

In [29]:
train.dtypes

Survived           int64
Pclass             int64
Sex                int64
Age              float64
SibSp              int64
Parch              int64
Fare             float64
Embarked           int64
Title              int64
Deck               int64
FamilySize         int64
AgeClass         float64
FarePerPerson    float64
dtype: object

In [30]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,FamilySize,AgeClass,FarePerPerson
PassengerId,,,,,,,,,,,,,
1,0,3,0,22.0,1,0,7.2500,1,0,0,1,66.0,3.62500
2,1,1,1,38.0,1,0,71.2833,2,1,1,1,38.0,35.64165
3,1,3,1,26.0,0,0,7.9250,1,2,0,0,78.0,7.92500
4,1,1,1,35.0,1,0,53.1000,1,1,1,1,35.0,26.55000
5,0,3,0,35.0,0,0,8.0500,1,0,0,0,105.0,8.05000


Uma vez que a tabela está pronta para análise, vamos criar a nossa coluna y.

In [31]:
train_y = train.Survived
train.drop(labels='Survived', axis=1, inplace=True)

Verificando a existência de NaNs

In [32]:
train.isnull().any()

Pclass           False
Sex              False
Age               True
SibSp            False
Parch            False
Fare             False
Embarked         False
Title            False
Deck             False
FamilySize       False
AgeClass          True
FarePerPerson    False
dtype: bool

In [33]:
train.Age.isnull().sum()

177

Substituindo os NaNs por 0 (zero).

In [34]:
train.Age.fillna(0, inplace=True)
train.AgeClass.fillna(0, inplace=True)
train.isnull().any().any()

False

<b>Importante!</b> Uma vez preparado os dados de treinamento, deve-se fazer o mesmo para os dados de teste. Deste modo, seguem-se os passos abaixo.

In [35]:
test = pd.read_csv('test.csv', index_col=0)
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [36]:
# criando uma nova coluna apenas com os títulos
test['Title'] = test['Name'].map(lambda x: substrings_in_string(x, title_list))
test['Title'] = test.apply(replace_titles, axis=1)

In [37]:
# preenchendo NaN com Unknown
test.Cabin.fillna('Unknown', inplace=True)
# elimiando os números das cabines
test['Deck'] = test['Cabin'].map(lambda x: substrings_in_string(x, cabin_list))

In [38]:
test['FamilySize'] = test.SibSp + test.Parch
test['AgeClass'] = test.Age*test.Pclass
test['FarePerPerson'] = test.Fare/(test.FamilySize + 1)

In [39]:
test.drop(labels=['Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

In [40]:
test.Sex = test.Sex.map({'male': 0, 'female': 1}).astype(int)
test.Embarked = test.Embarked.map({np.nan: 0, 'S': 1, 'C': 2, 'Q': 3}).astype(int)
test.Title = test.Title.map({'Mr': 0, 'Mrs': 1, 'Miss': 2, 'Master': 3}).astype(int)
test.Deck = test.Deck.map({'Unknown': 0, 'C': 1, 'E': 2, 'G': 3, 'D': 4, 'A': 5, 'B': 6, 'F': 7, 'T': 8}).astype(int)

In [41]:
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Deck,FamilySize,AgeClass,FarePerPerson
PassengerId,,,,,,,,,,,,
892,3,0,34.5,0,0,7.8292,3,0,0,0,103.5,7.829200
893,3,1,47.0,1,0,7.0000,1,1,0,1,141.0,3.500000
894,2,0,62.0,0,0,9.6875,3,0,0,0,124.0,9.687500
895,3,0,27.0,0,0,8.6625,1,0,0,0,81.0,8.662500
896,3,1,22.0,1,1,12.2875,1,1,0,2,66.0,4.095833


In [42]:
test.dtypes

Pclass             int64
Sex                int64
Age              float64
SibSp              int64
Parch              int64
Fare             float64
Embarked           int64
Title              int64
Deck               int64
FamilySize         int64
AgeClass         float64
FarePerPerson    float64
dtype: object

In [43]:
test.isnull().any()

Pclass           False
Sex              False
Age               True
SibSp            False
Parch            False
Fare              True
Embarked         False
Title            False
Deck             False
FamilySize       False
AgeClass          True
FarePerPerson     True
dtype: bool

In [44]:
test.Age.fillna(0, inplace=True)
test.AgeClass.fillna(0, inplace=True)
test.Fare.fillna(0, inplace=True)
test.FarePerPerson.fillna(0, inplace=True)
test.isnull().any().any()

False

Carregando variável y para test.

In [45]:
test_y = pd.read_csv('gender_submission.csv', index_col=0)
test_y.head()

,Survived
PassengerId,
892,0
893,1
894,0
895,0
896,1


## Aplicando os processos de Machine Learning

### KNN

O primeiro algorítimo a se utilizar será o KNN.

In [46]:
from sklearn.neighbors import KNeighborsClassifier

Primeiro irei utilizar o algorítimo no modo default, para depois variar os parâmetros.

In [47]:
KNN = KNeighborsClassifier()

In [48]:
KNN.fit(train, train_y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [49]:
KNN.score(test, test_y)

0.6578947368421053

Procurando pelo número K de vizinhos que maximiza o score.

In [50]:
s = time.time()
scoreMax = 0
for K in range(1,10):
    KNN = KNeighborsClassifier(n_neighbors=K)
    KNN.fit(train, train_y)
    score = KNN.score(test, test_y)
    if score > scoreMax:
        scoreMax = score
        Kmax = K
print('scoreMax[ K',Kmax,'] = %.5f' %scoreMax)
print('Tempo de execução: %.5f' %(time.time()-s))

scoreMax[ K 2 ] = 0.67225
Tempo de execução: 0.04131


### SVC

In [51]:
from sklearn.svm import SVC

Primeiro testanto no default.

In [52]:
svc = SVC()

In [53]:
svc.fit(train, train_y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [54]:
svc.score(test, test_y)

0.5956937799043063

Variando C e gamma.

In [55]:
s = time.time()
Gamma = np.arange(0.01,1,0.01)
c = np.arange(0.5, 2.5, 0.5)
scoreMax=0
for gamma in Gamma:
    for C in c:
        svc = SVC(C=C, gamma=gamma)
        svc.fit(train, train_y)
        score = svc.score(test, test_y)
        if score > scoreMax:
            scoreMax = score
            Cmax = C
            gammaMax = gamma
print('scoreMax [C:',Cmax,'gamma:',gammaMax,'] = %.5f' %scoreMax)
print('Tempo de execução: %.5f' %(time.time()-s))

scoreMax [C: 2.0 gamma: 0.01 ] = 0.67225
Tempo de execução: 19.76199


Estreitando a varredura dos parâmtros.

In [56]:
s = time.time()
Gamma = np.arange(0.001,0.02,0.001)
c = np.arange(1.0, 2.5, 0.05)
scoreMax=0
for gamma in Gamma:
    for C in c:
        svc = SVC(C=C, gamma=gamma)
        svc.fit(train, train_y)
        score = svc.score(test, test_y)
        if score > scoreMax:
            scoreMax = score
            Cmax = C
            gammaMax = gamma
print('scoreMax [C:',Cmax,'gamma:',gammaMax,'] = %.5f' %scoreMax)
print('Tempo de execução: %.5f' %(time.time()-s))

scoreMax [C: 2.450000000000001 gamma: 0.003 ] = 0.69139
Tempo de execução: 27.70339


Por algum motivo o modelo está preferindo o maior C possível. Irei variar apenas o valor de C para gamma = 0.003.

In [57]:
s = time.time()
c = np.arange(1, 10.5, 0.5)
scoreMax=0
for C in c:
    svc = SVC(C=C, gamma=0.003)
    svc.fit(train, train_y)
    score = svc.score(test, test_y)
    if score > scoreMax:
        scoreMax = score
        Cmax = C
        gammaMax = 0.003
print('scoreMax [C:',Cmax,'gamma:',gammaMax,'] = %.5f' %scoreMax)
print('Tempo de execução: %.5f' %(time.time()-s))

scoreMax [C: 8.5 gamma: 0.003 ] = 0.76794
Tempo de execução: 1.09238


Finalmente achamos um máximo para C.

Obtivemos um valor maior que o KNN. Agora irei trocar o kernel linear. Neste caso, como definido no site do sklearn, o parâmetro gamma não é válido para linear.

In [58]:
s = time.time()
scoreMax=0
c = np.arange(0.5, 10.5, 0.5)
for C in c:
    s_c = time.time()
    svc = SVC(C=C, kernel='linear')
    svc.fit(train, train_y)
    score = svc.score(test, test_y)
    if score > scoreMax:
        scoreMax = score
        Cmax = C
    print('scoreC [C:',C,'kernel: linear] = %.10f' %score)
    print('Tempo de execução parcial: %.5f' %(time.time()-s_c))
print('\n\tscoreMax [C:',Cmax,'kernel: linear] = %.5f' %scoreMax)
print('Tempo de execução: %.5f' %(time.time()-s))

scoreC [C: 0.5 kernel: linear] = 0.9521531100
Tempo de execução parcial: 3.66807
scoreC [C: 1.0 kernel: linear] = 0.9521531100
Tempo de execução parcial: 6.29049
scoreC [C: 1.5 kernel: linear] = 0.9521531100
Tempo de execução parcial: 15.71370
scoreC [C: 2.0 kernel: linear] = 0.9521531100
Tempo de execução parcial: 17.24569
scoreC [C: 2.5 kernel: linear] = 0.9521531100
Tempo de execução parcial: 16.73122
scoreC [C: 3.0 kernel: linear] = 0.9521531100
Tempo de execução parcial: 18.85131
scoreC [C: 3.5 kernel: linear] = 0.9521531100
Tempo de execução parcial: 20.85252
scoreC [C: 4.0 kernel: linear] = 0.9521531100
Tempo de execução parcial: 26.16999
scoreC [C: 4.5 kernel: linear] = 0.9521531100
Tempo de execução parcial: 27.21544
scoreC [C: 5.0 kernel: linear] = 0.9521531100
Tempo de execução parcial: 27.29249
scoreC [C: 5.5 kernel: linear] = 0.9521531100
Tempo de execução parcial: 27.21597
scoreC [C: 6.0 kernel: linear] = 0.9521531100
Tempo de execução parcial: 29.03415
scoreC [C: 6.5 ker

Nenhuma variação foi observada, porém o ganho no score ao mudar de kernel foi significativo. Deste modo seguiremos utilizando o SVC com o kernel linear, deixando os outros parâmetros como default.

Para tentar aumentar o score iremos fazer uma redução de dimensionalidade.

### PCA

In [59]:
from sklearn.decomposition import PCA

Como possuimos 12 colunas ao todo, iremos fazer um loop variando o número de componentes de 2 até 11.

In [60]:
s = time.time()
scoreMax = 0
for n_components in range(1,13):
    s_part = time.time()
    pca = PCA(n_components=n_components)
    train_pca = pca.fit_transform(train)
    test_pca = pca.transform(test)
    
    svc = SVC(kernel='linear')
    svc.fit(train_pca, train_y)
    score = svc.score(test_pca, test_y)
    if score > scoreMax:
        scoreMax = score
        nComp = n_components
    print('scorePart [ n', n_components,'] = ', score)
    print('Tempo de execução parcial: %.5f' %(time.time()-s_part))
print('\n\tscore [ n', nComp,'] = ', scoreMax)
print('Tempo de execução: %.5f' %(time.time()-s))

scorePart [ n 1 ] =  0.6411483253588517
Tempo de execução parcial: 0.43378
scorePart [ n 2 ] =  0.6363636363636364
Tempo de execução parcial: 7.11808
scorePart [ n 3 ] =  0.6291866028708134
Tempo de execução parcial: 5.08613
scorePart [ n 4 ] =  0.6291866028708134
Tempo de execução parcial: 4.15657
scorePart [ n 5 ] =  0.6363636363636364
Tempo de execução parcial: 4.56323
scorePart [ n 6 ] =  0.6339712918660287
Tempo de execução parcial: 8.23667
scorePart [ n 7 ] =  0.8803827751196173
Tempo de execução parcial: 1.33400
scorePart [ n 8 ] =  0.8779904306220095
Tempo de execução parcial: 1.92497
scorePart [ n 9 ] =  0.8875598086124402
Tempo de execução parcial: 10.17852
scorePart [ n 10 ] =  0.8971291866028708
Tempo de execução parcial: 5.49964
scorePart [ n 11 ] =  0.9521531100478469
Tempo de execução parcial: 11.95644
scorePart [ n 12 ] =  0.9521531100478469
Tempo de execução parcial: 11.47268

	score [ n 11 ] =  0.9521531100478469
Tempo de execução: 71.96171


A resolução para 11 ou 12 (máximo) dimensões apresentaram o mesmo resultado, sendo este o máximo obtido.

### Isomap

Foram feitos testes, mas o PC não aguentou rodar. Aparentemente ele entra num loop infinito utilizando 100% de processador, mas não 100% de memória.